In [1]:
import numpy as np
import os
from glob import glob
from tqdm import tqdm

In [2]:
os.getcwd()

'/home/jinjinjara1022/OmokAI'

In [3]:
base_path = '/home/jinjinjara1022/OmokAI/gomocup_results'
output_path = os.path.join('dataset3', os.path.basename(base_path))
os.makedirs(output_path, exist_ok=True)

In [4]:
file_list = glob(os.path.join(base_path, '%s*/*.psq' % ('Freestyle', )))
print(len(file_list))

48627


In [5]:
chk = 0
for index, file_path in enumerate(tqdm(file_list)):
    with open(file_path, 'r') as f:
        lines = f.read().splitlines()

    inputs, outputs = [], []
    board = np.zeros([19, 19], dtype=np.int8)
    save = True
    for i, line in enumerate(lines[1:]):
        if ',' not in line:
            break

        x, y, t = np.array(line.split(','), np.int8)
        
        if x == 20 or y == 20:
            save = False
            break

        if i % 2 == 0:
            player = 1
        else:
            player = 2

        input = board.copy().astype(np.int8)
        input[(input != player) & (input != 0)] = -1
        input[(input == player) & (input != 0)] = 1
        
        output = np.zeros([19, 19], dtype=np.int8)
        output[y-1, x-1] = 1

        for k in range(4):
            input_rot = np.rot90(input, k=k)
            output_rot = np.rot90(output, k=k)

            inputs.append(input_rot)
            outputs.append(output_rot)

            inputs.append(np.fliplr(input_rot))
            outputs.append(np.fliplr(output_rot))

            inputs.append(np.flipud(input_rot))
            outputs.append(np.flipud(output_rot))
        
        board[y-1, x-1] = player

    if save:
        chk += 1
        np.savez_compressed(os.path.join(output_path, '%s.npz' % (str(chk).zfill(5))), inputs=inputs, outputs=outputs)

100%|████████████████████████████████████| 48627/48627 [05:00<00:00, 161.67it/s]
